# Ingestão Bronze - Controle Ativo

Este notebook realiza a leitura de um arquivo CSV contendo o controle de Ações e FII
e grava os dados em formato Delta na camada **Bronze**.

**Origem dos dados:**
- Arquivo: `D:\Projetos\Jornada_financas_pessoais\data\source\cadastro_cliente.xlsx`

**Destino:**
- Caminho Delta: `D:\Projetos\Jornada_financas_pessoais\data\delta\bronze\raw_cadcliente`
- Tabela Delta: `bronze.raw_cadcliente`


## Imports

In [1]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
import pandas as pd
import glob
import os

## Start Spark Session

In [2]:
# Inicializa uma SparkSession
builder = SparkSession.builder \
    .appName("Leitura de Arquivo de Formato Fixo") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado!\n")


✅ Spark 3.5.7 iniciado!



## Variables

In [4]:
# Define o caminho do diretório dos arquivos de origem
source_path = "D:/Projetos/Jornada_financas_pessoais/data/source"

# Define caminhos locais onde serão armazenadas as tabelas Delta
base_bronze_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze"

# Define o caminho da tabela Delta Source
delta_path_controleativo = f"{base_bronze_path}/raw_controleativo"

## Leitura do arquivo Excel

In [8]:
# Lista todos os arquivos Excel que começam com "Controle de Ações e FII"
file_list = glob.glob(f'{source_path}/Controle de Ações e FII*.xlsx')

print(f"Arquivos Excel encontrados: {len(file_list)}")
for f in file_list:
    print(f"  - {os.path.basename(f)}")

# Lê todos os arquivos Excel e adiciona coluna com nome do arquivo
dfs_list = []

for file_path in file_list:
    print(f"\nProcessando: {os.path.basename(file_path)}")
    
    # Lê apenas a aba "FII"
    # header=0 (padrão) usa a primeira linha como nome das colunas
    df_temp = pd.read_excel(
        file_path,
        sheet_name='FII',
        engine='openpyxl',
        header=0,  # Primeira linha (índice 0) = cabeçalho (PADRÃO)
        dtype=str  # Força todas as colunas como string
    )
    
    # Adiciona coluna com nome do arquivo (apenas o nome, sem path)
    df_temp['arquivo_origem'] = os.path.basename(file_path)
    
    dfs_list.append(df_temp)
    print(f"  ✓ {len(df_temp)} registros lidos")

 # Mostra as primeiras linhas
print(dfs_list)

# Converte para PySpark DataFrame
#df_controleativo = spark.createDataFrame(dfs_list)


Arquivos Excel encontrados: 1
  - Controle de Ações e FII - Taynara.xlsx

Processando: Controle de Ações e FII - Taynara.xlsx
  ✓ 63 registros lidos
[                   DATA   PAPEL OP QTD   PREÇO CUS.T. TOTAL OP  RAT.  \
0   2024-09-23 00:00:00  AFHI11  C   1   96.83   1.11    96.83  0.03   
1   2024-09-23 00:00:00  AFHI11  C   5   96.84   1.11    484.2  0.15   
2   2024-09-23 00:00:00  IRDM11  C   4   68.08   1.11   272.32  0.08   
3   2024-09-23 00:00:00  KNSC11  C  40    9.23   1.11    369.2  0.11   
4   2024-09-23 00:00:00  RZTR11  C   4    91.1   1.11    364.4  0.11   
..                  ...     ... ..  ..     ...    ...      ...   ...   
58  2025-04-29 00:00:00  VGIA11  C  10    8.73   0.66     87.3  0.02   
59  2025-04-29 00:00:00  AFHI11  C   7    92.8   0.66    649.6   0.2   
60  2025-04-29 00:00:00  AFHI11  C   1   92.79   0.66    92.79  0.02   
61  2025-04-29 00:00:00  IRDM11  C  10   68.42   0.66    684.2  0.21   
62  2025-04-29 00:00:00  TRXF11  C   6  103.32   0.66   61

## Save Data Frame

In [ ]:
# Grava em formato Delta
(
    df_controleativo.write.format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "false")
    .save(delta_path_cadcliente)
)

print(f"[SUCESSO] DataFrame gravado no Delta Lake em: {delta_path_cadcliente}")

[SUCESSO] DataFrame gravado no Delta Lake em: D:/Projetos/Jornada_financas_pessoais/data/delta/bronze/raw_cadcliente


## Stop Spark Session

In [16]:
# Encerra a SparkSession
spark.stop()